In [56]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.optimize import curve_fit
from pylatexenc.latex2text import LatexNodes2Text

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
df = pd.read_csv("./data/regular/phase/size=100_step=0.001_max=2.55_seed=1/data.csv")
df

,t,n,T,M,E
0,1.0,1.0,0.0001,1.0000,-40000.0
1,2.0,1.0,0.0011,1.0000,-40000.0
2,3.0,1.0,0.0021,1.0000,-40000.0
3,4.0,1.0,0.0031,1.0000,-40000.0
4,5.0,1.0,0.0041,1.0000,-40000.0
...,...,...,...,...,...
2315,96202.0,47.0,2.3151,0.2008,-32808.0
2316,96246.0,44.0,2.3161,0.2132,-33080.0
2317,96471.0,225.0,2.3171,0.2158,-33424.0
2318,96533.0,62.0,2.3181,0.0398,-33124.0


In [226]:
def calc_regression(df: pd.DataFrame) -> (pd.DataFrame, list[float]):
    MIN_M = 0.1
    BOUNDS = (1e-12, 5)
    
    def mt_fit(t: float, m0: float, tc: float, b: float) -> float:
        v = (1 - t/tc)
        return m0 * np.sign(v) * np.float_power(np.abs(v), b)

    def slice_data(df: pd.DataFrame) -> pd.DataFrame:
        return df[:np.max((df["M"] > MIN_M).argmin() - 1, 0)]

    data = slice_data(df)
    popt, _ = curve_fit(mt_fit, data["T"], data["M"], bounds=BOUNDS, maxfev=1e12) 

    return (pd.concat([
        data[["T"]], 
        data[["T"]].map(lambda t: mt_fit(t, *popt)).rename(columns={"T": "M"})
    ], axis=1), popt)
    

In [235]:
def plot_phase(path: str) -> alt.Chart:
    df = pd.read_csv(path)
    fit_df, fit_props = calc_regression(df)
    
    base: alt.Chart = (
        alt.Chart(df)
            .mark_circle()
            .encode(
                alt.X("T:Q").title("k_BT/J"),
                alt.Y("M:Q"),
                alt.Color("E:Q"),
                opacity=alt.value(0.5),
            )
    )

    fit: alt.Chart = (
        alt.Chart(fit_df)
            .mark_line(strokeDash=[6,4])
            .encode(
                alt.X("T:Q"),
                alt.Y("M:Q"),
                color=alt.value("#1b1b1b")
            )
    )

    tc: alt.Chart = (
        alt.Chart(pd.DataFrame({"T": [fit_props[1]]}))
            .mark_rule()
            .encode(
                x="T:Q"
            )
    )

    tc_label: alt.Chart = (
        tc
            .transform_calculate(
                TC="'T_C='+format(datum.T, '.4f')"
            )
            .mark_text(
                baseline="middle",
                align="right",
                dx=-5,
                fontWeight=700
            )
            .encode(
                alt.Text("TC:N")
            )
    )

    return base + fit + tc + tc_label

def plot_phase_cmp(size=100, step=0.001, max=2.55, seed=1) -> alt.Chart:
    def path_of_type(type: str) -> str:
        return f"./data/{type}/phase/size={size}_step={step}_max={max}_seed={seed}/data.csv"

    def plot_sub(type: str) -> alt.Chart:
        return plot_phase(path_of_type(type)).properties(title=type)
        
    return (plot_sub("regular") | plot_sub("irregular")).properties(
        title=alt.TitleParams(f"Size={size:.2E}, step={step:.2E}, max={max} seed={seed}", anchor="middle")
    )

In [236]:
plot_phase_cmp(seed=1) & plot_phase_cmp(seed=100)

alt.VConcatChart(...)